In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

In [2]:
import geopandas as gpd
import pandas as pd
from siuba import *
import numpy as np

from segment_speed_utils import helpers, gtfs_schedule_wrangling
from shared_utils import rt_dates, gtfs_utils_v2
import folium
import itertools

In [3]:
from update_vars import (analysis_date, AM_PEAK, PM_PEAK, EXPORT_PATH, GCS_FILE_PATH, PROJECT_CRS,
SEGMENT_BUFFER_METERS, AM_PEAK, PM_PEAK, HQ_TRANSIT_THRESHOLD, MS_TRANSIT_THRESHOLD)

In [4]:
from calitp_data_analysis.tables import tbls

# Why is the Coronado ferry missing from our datasets?

See https://github.com/cal-itp/data-analyses/issues/1458

In [5]:
analysis_date

'2025-04-16'

In [6]:
import rail_ferry_brt_stops

In [7]:
import importlib
importlib.reload(rail_ferry_brt_stops)

<module 'rail_ferry_brt_stops' from '/home/jovyan/data-analyses/high_quality_transit_areas/rail_ferry_brt_stops.py'>

In [104]:
stops_all = rail_ferry_brt_stops.assemble_stops(analysis_date)

In [124]:
ferry = rail_ferry_brt_stops.grab_ferry_stops(stops_all)

In [125]:
# ferry # coronado now present!

,feed_key,stop_id,stop_name,geometry,name,route_id,route_type,route_desc,hqta_type
0,8510daa6c8576e648fcbd4f92ea73a51,99440,Broadway Pier,POINT (265337.654 -584519.841),San Diego Schedule,COR,4,None,major_stop_ferry
1,8510daa6c8576e648fcbd4f92ea73a51,99439,Coronado Ferry Landing,POINT (265739.278 -586372.329),San Diego Schedule,COR,4,None,major_stop_ferry
2,1dc19b0c7c704e7663492625b4c1acb1,1,Treasure Island,POINT (-208524.906 -19590.082),Bay Area 511 Treasure Island Ferry Schedule,TISF,4,None,major_stop_ferry
3,1dc19b0c7c704e7663492625b4c1acb1,2,San Francisco,POINT (-210370.936 -21759.606),Bay Area 511 Treasure Island Ferry Schedule,TISF,4,None,major_stop_ferry
4,8510daa6c8576e648fcbd4f92ea73a51,99441,5th Av Pier,POINT (266488.380 -585759.477),San Diego Schedule,COR,4,None,major_stop_ferry
5,7bc8ee29bf0fd457367c7ae3dcdcb2c1,72011,San Francisco Ferry Building Gate E,POINT (-210291.852 -21944.196),Bay Area 511 San Francisco Bay Ferry Schedule,VJO,4,None,major_stop_ferry
6,7bc8ee29bf0fd457367c7ae3dcdcb2c1,72011,San Francisco Ferry Building Gate E,POINT (-210291.852 -21944.196),Bay Area 511 San Francisco Bay Ferry Schedule,RCH,4,None,major_stop_ferry
7,7bc8ee29bf0fd457367c7ae3dcdcb2c1,72011,San Francisco Ferry Building Gate E,POINT (-210291.852 -21944.196),Bay Area 511 San Francisco Bay Ferry Schedule,OA,4,None,major_stop_ferry
8,7bc8ee29bf0fd457367c7ae3dcdcb2c1,72011,San Francisco Ferry Building Gate E,POINT (-210291.852 -21944.196),Bay Area 511 San Francisco Bay Ferry Schedule,HB,4,None,major_stop_ferry
9,7bc8ee29bf0fd457367c7ae3dcdcb2c1,72011,San Francisco Ferry Building Gate E,POINT (-210291.852 -21944.196),Bay Area 511 San Francisco Bay Ferry Schedule,SEA,4,None,major_stop_ferry


## earlier stages

In [69]:
trips = helpers.import_scheduled_trips(
    analysis_date,
    columns = [
        "name", "feed_key",
        "trip_instance_key", "trip_id", 
        "route_id", "route_type", "route_desc",
        "shape_id"
    ],
    get_pandas = True
)

### present in trips

In [73]:
trips.query('route_id == "COR"').head(3) #  present in trips

,name,feed_key,trip_instance_key,trip_id,route_id,route_type,route_desc,shape_id
8010,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,bb077aa05b3bab38c8d87f16ec567a6f,15331358,COR,4,None,None
8011,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,ae163073f5df72dd20e25eef248e85b8,15331347,COR,4,None,None
8012,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,a9e6ebfead8f4d083d5e2c352a667af0,15331420,COR,4,None,None


In [51]:
stop_times = helpers.import_scheduled_stop_times(
    analysis_date,
    columns = ["feed_key", "schedule_gtfs_dataset_key",
               "stop_id", "trip_instance_key"],
    with_direction = True,
    get_pandas = True
)

In [53]:
stops_with_route = pd.merge(
    stop_times,
    trips,
    on = "trip_instance_key",
    how = "inner"
).drop(
    columns = "trip_instance_key"
).drop_duplicates().reset_index(drop=True)

### not present in stop times with direction

In [54]:
stops_with_route.query('route_id == "COR"') #  gone

,feed_key_x,schedule_gtfs_dataset_key,stop_id,name,feed_key_y,trip_id,route_id,route_type,route_desc


## present in no-direction version of stop times

In [110]:
st_no_dir = helpers.import_scheduled_stop_times(
    analysis_date,
    columns = ["feed_key", "trip_id",
               "stop_id",
               # "pickup_type", "drop_off_type" #  these aren't here...
              ],
    with_direction = False,
    get_pandas = True
)

In [111]:
from siuba import *

In [116]:
df = (tbls.mart_gtfs.dim_stop_times()
>> filter(_.stop_id == '65866', _.feed_key == '186750f1308db1f41f43fe7d661981ba') >> collect())

In [118]:
df >> select(_.feed_key, _.stop_id, _.pickup_type, _.drop_off_type)

,feed_key,stop_id,pickup_type,drop_off_type
0,186750f1308db1f41f43fe7d661981ba,65866,0,0
1,186750f1308db1f41f43fe7d661981ba,65866,0,0
2,186750f1308db1f41f43fe7d661981ba,65866,0,0
3,186750f1308db1f41f43fe7d661981ba,65866,0,0
4,186750f1308db1f41f43fe7d661981ba,65866,0,0
...,...,...,...,...
126,186750f1308db1f41f43fe7d661981ba,65866,0,0
127,186750f1308db1f41f43fe7d661981ba,65866,0,0
128,186750f1308db1f41f43fe7d661981ba,65866,0,0
129,186750f1308db1f41f43fe7d661981ba,65866,0,0


In [113]:
trips.query('route_id == "Green Line"').merge(st_no_dir, on=['feed_key', 'trip_id']).head(3)

,name,feed_key,trip_instance_key,trip_id,route_id,route_type,route_desc,shape_id,stop_id
0,Bay Area 511 Santa Clara Transit Schedule,186750f1308db1f41f43fe7d661981ba,5a9af219cc5c11db32d07657d18887ef,3732157,Green Line,0,None,116453,65379
1,Bay Area 511 Santa Clara Transit Schedule,186750f1308db1f41f43fe7d661981ba,5a9af219cc5c11db32d07657d18887ef,3732157,Green Line,0,None,116453,65378
2,Bay Area 511 Santa Clara Transit Schedule,186750f1308db1f41f43fe7d661981ba,5a9af219cc5c11db32d07657d18887ef,3732157,Green Line,0,None,116453,65377


In [91]:
trips.query('route_id == "COR"').merge(st_no_dir, on=['feed_key', 'trip_id']).head(3)

,name,feed_key,trip_instance_key,trip_id,route_id,route_type,route_desc,shape_id,stop_id
0,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,bb077aa05b3bab38c8d87f16ec567a6f,15331358,COR,4,None,None,99440
1,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,bb077aa05b3bab38c8d87f16ec567a6f,15331358,COR,4,None,None,99439
2,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,ae163073f5df72dd20e25eef248e85b8,15331347,COR,4,None,None,99440


## check `../gtfs_funnel/stop_times_with_direction.py`

* not a problem with that script, but it only works for trips with a `shape_id`...
* the Coronado Ferry has none, so it's not included in directional stop times
* we don't actually need direction here, solved by using the non-directional stop times instead

In [55]:
import sys
sys.path.append('../gtfs_funnel')

In [59]:
import stop_times_with_direction as st_dir

In [60]:
st_prepped = st_dir.prep_scheduled_stop_times(analysis_date)

In [71]:
trips.query('route_id == "COR"').merge(st_prepped, on=['trip_instance_key']).head(3) #  OK here

,name,feed_key_x,trip_instance_key,trip_id,route_id,route_type,route_desc,shape_id,feed_key_y,stop_id,stop_sequence,schedule_gtfs_dataset_key,shape_array_key,stop_name,geometry
0,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,bb077aa05b3bab38c8d87f16ec567a6f,15331358,COR,4,None,None,8510daa6c8576e648fcbd4f92ea73a51,99439,2,baeeb157e85a901e47b828ef9fe75091,None,Coronado Ferry Landing,POINT (265739.278 -586372.329)
1,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,bb077aa05b3bab38c8d87f16ec567a6f,15331358,COR,4,None,None,8510daa6c8576e648fcbd4f92ea73a51,99440,1,baeeb157e85a901e47b828ef9fe75091,None,Broadway Pier,POINT (265337.654 -584519.841)
2,San Diego Schedule,8510daa6c8576e648fcbd4f92ea73a51,ae163073f5df72dd20e25eef248e85b8,15331347,COR,4,None,None,8510daa6c8576e648fcbd4f92ea73a51,99439,2,baeeb157e85a901e47b828ef9fe75091,None,Coronado Ferry Landing,POINT (265739.278 -586372.329)


## final output check

* looks good!

In [8]:
path = f'{GCS_FILE_PATH}hqta_areas.parquet'

In [9]:
gdf = gpd.read_parquet(path)

In [10]:
gdf.head(3)

,agency_primary,agency_secondary,hqta_type,hqta_details,route_id,base64_url_primary,base64_url_secondary,org_id_primary,org_id_secondary,geometry
0,Alameda-Contra Costa Transit District,None,hq_corridor_bus,stop_along_hq_bus_corridor_single_operator,10,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,"MULTIPOLYGON (((-122.13315 37.69219, -122.1334..."
1,Alameda-Contra Costa Transit District,None,hq_corridor_bus,stop_along_hq_bus_corridor_single_operator,14,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,"MULTIPOLYGON (((-122.22584 37.79946, -122.2256..."
2,Alameda-Contra Costa Transit District,None,hq_corridor_bus,stop_along_hq_bus_corridor_single_operator,18,aHR0cHM6Ly9hcGkuNTExLm9yZy90cmFuc2l0L2RhdGFmZW...,None,recOZgevYf7Jimm9L,None,"MULTIPOLYGON (((-122.30800 37.89073, -122.3079..."


In [9]:
# gdf.query('hqta_type == "major_stop_ferry"').explore()